In [1]:
from src.RawDataLoaders import MS_Marco_RawDataLoader, CAsT_RawDataLoader
from src.pipe_datasets import Manual_Query_BM25_Reranking_Dataset, Reranking_Validation_Dataset, Manual_Query_RUN_File_Reranking_Dataset
from src.models_and_transforms.run_file_models import Run_File_Searcher
from src.models_and_transforms.BERT_models import BERT_Reranker
from src.models_and_transforms.Longformer_models import Longformer_Reranker
from src.models_and_transforms.BM25_models import BM25_Ranker
from src.Experiments import CAsT_experiment, Ranking_Experiment, RUN_File_Transform_Exporter
from src.trainers import Model_Trainer
from src.models_and_transforms.complex_transforms import BERT_Score_Transform, BERT_ReRanker_Transform, BM25_Search_Transform, \
                                                         Oracle_ReRanker_Transform, RUN_File_Search_Transform, monoBERT_Scorer_Transform, \
                                                         MonoBERT_ReRanker_Transform, DuoBERT_Scorer_Transform, DuoBERT_ReRanker_Transform
from src.models_and_transforms.text_transforms import Query_Resolver_Transform, Document_Resolver_Transform, Reranking_Flattener_Transform, \
                                                      MonoBERT_Numericalise_Transform, BERT_Denumericalise_Transform,  \
                                                      DuoBERT_Numericalise_Transform, Relevance_Model_Transform, Simple_Query_Expansion_Transform

from transformers import LongformerConfig, LongformerModel, LongformerTokenizer, BertTokenizer, BertModel, BertForSequenceClassification
from pytorch_lightning import Trainer, Callback, seed_everything
import pickle
import random
import numpy as np
import os
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import torch
import jsonlines
seed_everything(42)

from tqdm.auto import tqdm 
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [3]:
raw_data_loader = MS_Marco_RawDataLoader(from_pickle=True)
get_query_fn = raw_data_loader.get_query
get_doc_fn = raw_data_loader.get_doc

In [4]:
train_raw_samples = raw_data_loader.get_topics("train")

In [5]:
searcher = Run_File_Searcher("saved_models/MARCO_train_BM25_full.run")

KeyboardInterrupt: 

#### Removing probllematic samples

In [6]:
bad_samples = [sample for sample in train_raw_samples if sample['q_id'] not in searcher.query_doc_mapping]

In [7]:
for sample in bad_samples:
    print(sample)
    train_raw_samples.remove(sample)

{'q_id': '140329', 'q_rel': ['MARCO_6542451']}
{'q_id': '1078982', 'q_rel': ['MARCO_4115897']}
{'q_id': '502557', 'q_rel': ['MARCO_1271975']}
{'q_id': '48509', 'q_rel': ['MARCO_2063851']}
{'q_id': '56573', 'q_rel': ['MARCO_3198289']}
{'q_id': '129844', 'q_rel': ['MARCO_7817031']}
{'q_id': '197820', 'q_rel': ['MARCO_5510763']}
{'q_id': '522517', 'q_rel': ['MARCO_3075528']}
{'q_id': '205266', 'q_rel': ['MARCO_5143713']}


In [10]:
len(bad_samples)

0

In [8]:
train_dataset = Manual_Query_RUN_File_Reranking_Dataset(train_raw_samples, get_query_fn, get_doc_fn, "saved_models/MARCO_train_BM25_full.run", hits=100, num_neg_samples=50)

In [35]:
train_dataset[50000]

{'q_id': '502744',
 'q_rel': ['MARCO_5338628'],
 'query': 'starter remove on a polaris 400 sportsman',
 'd_id': 'MARCO_5338630',
 'label': 0,
 'doc': "polaris 400 starter. Follow polaris 400 starter to get e-mail alerts and updates on your eBay Feed. Unfollow polaris 400 starter to stop getting updates on your eBay Feed.Yay! You're now following polaris 400 starter in your eBay Feed.ollow polaris 400 starter to get e-mail alerts and updates on your eBay Feed. Unfollow polaris 400 starter to stop getting updates on your eBay Feed. Yay! You're now following polaris 400 starter in your eBay Feed.",
 'input_text': "starter remove on a polaris 400 sportsman [SEP] polaris 400 starter. Follow polaris 400 starter to get e-mail alerts and updates on your eBay Feed. Unfollow polaris 400 starter to stop getting updates on your eBay Feed.Yay! You're now following polaris 400 starter in your eBay Feed.ollow polaris 400 starter to get e-mail alerts and updates on your eBay Feed. Unfollow polaris 400

In [ ]:
train_dataset = Manual_Query_BM25_Reranking_Dataset(train_raw_samples[:20000], get_query_fn, get_doc_fn, hits=150, num_neg_samples=100, index_dir="datasets/MS_MARCO/MARCO_anserini")

In [ ]:
train_dataloader = train_dataset.to_dataloader(8, num_workers=32, shuffle=True)

In [7]:
model = Longformer_Reranker()

In [17]:
torch.save(model.state_dict(), "saved_models/Longformer/Longformer_ReRanker_MARCO.ckpt")

In [30]:
model.load_state_dict(torch.load("saved_models/Longformer/Longformer_ReRanker_MARCO.ckpt"))

<All keys matched successfully>

In [53]:
samples = [{"input_ids":[55,66,33]},{"input_ids":[45,76,33]}]
score_transform = BERT_Score_Transform("saved_models/BERT_ReRanker_MARCO.ckpt")
score_transform(samples)

<All keys matched successfully>
BERT ReRanker initialised on cuda. Batch size 64


[{'input_ids': [55, 66, 33], 'score': 0.044405270367860794},
 {'input_ids': [45, 76, 33], 'score': 0.044405270367860794}]

In [132]:
samples = [{'q_id':"121352","query":"define extreme", 'search_results':[('MARCO_6237152', 0.6), ('MARCO_2912794', 0.6)]}]
rerank_transform = BERT_ReRanker_Transform("saved_models/Longformer/Longformer_ReRanker_MARCO.ckpt", get_doc_fn)
rerank_transform(samples)

<All keys matched successfully>
BERT ReRanker initialised on cuda. Batch size 64


[{'q_id': '121352',
  'query': 'define extreme',
  'search_results': [('MARCO_6237152', 0.6), ('MARCO_2912794', 0.6)],
  'reranked_results': [('MARCO_2912794', 0.992748498916626),
   ('MARCO_6237152', -0.01869625225663185)]}]

In [8]:
valid_q_rels = raw_data_loader.q_rels("dev")
valid_raw_samples = raw_data_loader.get_topics("dev")[:400]
valid_samples = Query_Resolver_Transform(get_query_fn)(valid_raw_samples)
valid_BM25_results = RUN_File_Search_Transform('saved_models/MARCO_dev_BM25.run', hits=100)(valid_raw_samples)
val_dataset = Reranking_Validation_Dataset(valid_BM25_results, get_query_fn, get_doc_fn)
val_dataloader = val_dataset.to_dataloader(128, num_workers=32, shuffle=False)

In [12]:
# experiment
expr = Ranking_Experiment(valid_q_rels)
print("BM25 results")
print(expr(valid_BM25_results))

print("ORACLE results")
valid_oracle_rerank_results = Oracle_ReRanker_Transform(valid_q_rels)(valid_BM25_results)
for sample in valid_oracle_rerank_results:
    sample["search_results"] = sample["reranked_results"]
print(expr(valid_oracle_rerank_results))

print("BERT+BM25 results")
valid_BERT_rerank_results = BERT_ReRanker_Transform("saved_models/BERT_reranker_q100k_h100_checkpoints/BERT_ReRanker_MARCO_from_valid_0.38551378521302254.ckpt", get_doc_fn, device="cuda:2", batch_size=256)(valid_BM25_results)
for sample in valid_BERT_rerank_results:
    sample["search_results"] = sample["reranked_results"]
print(expr(valid_BERT_rerank_results))

BM25 results
{'map': 0.18098515570583398, 'recip_rank': 0.18098515570583398, 'ndcg': 0.27305565934364895, 'set_recall': 0.645}
ORACLE results
{'map': 0.645, 'recip_rank': 0.645, 'ndcg': 0.645, 'set_recall': 0.645}
BERT+BM25 results
cpu
<All keys matched successfully>
BERT ReRanker initialised on cuda:2. Batch size 256



{'map': 0.3138712999797035, 'recip_rank': 0.3138712999797035, 'ndcg': 0.38551378521302254, 'set_recall': 0.645}


In [17]:
expr = Ranking_Experiment(valid_q_rels)
print(expr(valid_BERT_rerank_results))

TypeError: Unable to resolve all measures.

In [1]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [32]:
my_trainer = Model_Trainer(gpus=[0])
my_trainer.train(model, train_dataloader)

Detected 8 GPUS available, using [0].
Main device is: cuda:0



Keyboard Interrupt!


{'train_loss': [tensor(0.7658, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.5080, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.3244, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.3060, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.2623, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.4784, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.5412, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.4063, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.4163, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.5151, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.3588, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.4145, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.3521, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.2835, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.3656, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.5360, device='cuda:0', grad_fn=<MseLossBackwa

In [9]:
checkpoint_callback = ModelCheckpoint(
    filepath='saved_models/BERT_reranker_q500k_h150_checkpoints/test_saves/',
    save_top_k=3,
    verbose=True,
    monitor='ndcg',
    mode='min',
    prefix='BERT_reranker_500k_queries'
)

In [10]:
model.set_validation_q_rels(valid_q_rels)

In [11]:
wandb_logger = WandbLogger(name='Longformer2k',project='pytorchlightning')

In [16]:
trainer = Trainer(gpus=1, profiler=True, 
                  print_nan_grads=True, 
                  num_sanity_val_steps=0,#len(val_dataloader), 
                  val_check_interval=0.02,
                  accumulate_grad_batches=1,
                  logger= wandb_logger,
                  checkpoint_callback=checkpoint_callback)
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type            | Params
-----------------------------------------------
0 | Longformer | LongformerModel | 148 M 
1 | dropout    | Dropout         | 0     
2 | proj_layer | Linear          | 769   


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_train_start      	|  0.014943       	|  0.014943       
on_epoch_start      	|  0.0012078      	|  0.0012078      
get_train_batch     	|  0.0045961      	|  13.908         
on_batch_start      	|  1.1327e-05     	|  0.034276       
model_forward       	|  0.71043        	|  2149.7         
model_backward      	|  0.45656        	|  1381.1         
on_after_backward   	|  2.2378e-06     	|  0.0067693      
optimizer_step      	|  0.017894       	|  54.129         
on_batch_end        	|  0.0018957      	|  5.7344         
on_train_end        	|  0.0012826      	|  0.0012826      



1

In [20]:
trainer.save_checkpoint("example.ckpt")

In [ ]:
%debug

In [36]:
outputs = model.validation_step(batch, 0)

In [37]:
outputs["valid_outputs"]

[0.64140784740448,
 0.650149941444397,
 0.6211758255958557,
 0.650149941444397,
 0.6418268084526062,
 0.650149941444397,
 0.6800584197044373,
 0.650149941444397,
 0.6111412048339844,
 0.650149941444397,
 0.6355796456336975,
 0.650149941444397,
 0.6755321025848389,
 0.650149941444397,
 0.6149528622627258,
 0.650149941444397]

In [148]:
train_raw_sampleget_topicsw_data_loader.get_topics("train")
train_dataset = Manual_Query_BM25_Reranking_Dataset(train_raw_samples, get_query_fn, get_doc_fn, hits=100)
train_dataloader = train_dataset.to_dataloader(2, num_workers=32)

NameError: name 'train_raw_sampleget_topicsw_data_loader' is not defined

In [41]:
BERT_BM25_reranker = BERT_BM25_Reranker(raw_data_loader.get_doc, raw_data_loader.get_query)

In [43]:
reranking_DataProcessor = Resolved_Query_Reranking_DataProcessor(raw_data_loader.get_doc, 
                                                  raw_data_loader.get_query, 
                                                  raw_data_loader.get_topics("train"), 
                                                  BERT_BM25_reranker.first_pass_model.predict, 
                                                  numericalizer,
                                                  max_length=512)

In [7]:
dataloader = reranking_DataProcessor.to_dataloader(2, num_workers=0)

In [96]:
len(train_dataset)

25352

In [147]:
for batch in tqdm.tqdm(train_dataloader):
    print(batch["input_ids"])
    break

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/nfs/phd_by_carlos/notebooks/src/DataProcessors.py", line 493, in __getitem__
    samples = transform(samples)
  File "/nfs/phd_by_carlos/notebooks/src/DataProcessors.py", line 465, in __call__
    samples = transform(samples)
  File "/nfs/phd_by_carlos/notebooks/src/DataProcessors.py", line 421, in __call__
    sample_obj["doc"] = self.get_doc_fn(sample_obj["d_id"])
  File "/nfs/phd_by_carlos/notebooks/src/dat

In [41]:
BM25_Ranker(get_query_fn).predict("32_4")

'BM25_Ranker'

In [9]:
def eval_call(model):
    print("called evaluation")

In [10]:
model = Longformer_Reranker()

In [11]:
model.eval_callback = eval_call

In [45]:
trainer = Trainer(gpus=[0], profiler=True, gradient_clip_val=0.5, distributed_backend='dp', check_val_every_n_epoch=1)
trainer.fit(model, dataloader)

NameError: name 'dataloader' is not defined

In [10]:
dataloader = torch.utils.data.DataLoader(torch.tensor([]), batch_size=1)

# Running an experiment on Y2 data

In [12]:
CAsT_raw_data_loader = CAsT_RawDataLoader()
get_query_fn = CAsT_raw_data_loader.get_query
get_doc_fn = CAsT_raw_data_loader.get_doc
eval_raw_samples = CAsT_raw_data_loader.get_topics("train")
CAsT_q_rels = CAsT_raw_data_loader.q_rels

In [13]:
eval_samples = Query_Resolver_Transform(get_query_fn, utterance_type="manual_rewritten_utterance")(eval_raw_samples)
eval_BM25_results = BM25_Search_Transform(index_dir='datasets/TREC_CAsT/CAsT_collection_with_meta.index', hits=500)(eval_samples)

In [32]:
# experiment
expr = Ranking_Experiment(CAsT_q_rels)
print("BM25 results")
print(expr(eval_BM25_results))

print("RUN File Model")
eval_run_rerank_results = RUN_File_Search_Transform('saved_models/CAsT_y1_pgbert.run', hits=500)(eval_samples)
print(expr(eval_run_rerank_results))

print("ORACLE results")
eval_oracle_rerank_results = Oracle_ReRanker_Transform(CAsT_q_rels)(eval_BM25_results)
for sample in eval_oracle_rerank_results:
    sample["search_results"] = sample["reranked_results"]
print(expr(eval_oracle_rerank_results))

print("BERT+BM25 results")
eval_BERT_rerank_results = BERT_ReRanker_Transform("saved_models/BERT_reranker_q100k_h100_checkpoints/BERT_ReRanker_MARCO_from_valid_0.38551378521302254.ckpt", get_doc_fn, device="cuda:2", batch_size=256)(eval_BM25_results)
for sample in eval_BERT_rerank_results:
    sample["search_results"] = sample["reranked_results"]
print(expr(eval_BERT_rerank_results))

BM25 results
{'map': 0.19679730164325712, 'recip_rank': 0.4596050256759099, 'ndcg_cut_3': 0.306364841910461, 'set_recall': 0.7744165761018662}
RUN File Model



{'map': 0.22648101607514953, 'recip_rank': 0.48689326704789326, 'ndcg_cut_3': 0.3621256024784965, 'set_recall': 0.5633517089242985}
ORACLE results
{'map': 0.5633517089242985, 'recip_rank': 0.8345864661654135, 'ndcg_cut_3': 0.7816506828787912, 'set_recall': 0.5633517089242985}
BERT+BM25 results
cpu
<All keys matched successfully>
BERT ReRanker initialised on cuda:2. Batch size 256



{'map': 0.10678501989112972, 'recip_rank': 0.25388222588499865, 'ndcg_cut_3': 0.14453453816167808, 'set_recall': 0.5633517089242985}


In [9]:
get_doc_fn('MARCO_5593358')

'â\x80\x8bâ\x80\x8bResidential garage doors from Overhead Door are among the most dependable in the industry, so you can feel good knowing that weâ\x80\x99ll be there â\x80\x94 day or night, winter or summer. For added peace of mind, our home garage doors have also been proven to be durable and long lasting. The reliability of your garage door will help you stay on schedule in the morning. Its beauty will greet you at the end of a busy workday, opening convenient, comfortable passage to your home. And through the night, the security of your garage door will help you rest assured that your family is safe.'

In [79]:
class BertForPassageRanking(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.weight = torch.autograd.Variable(torch.ones(2, config.hidden_size),
                                              requires_grad=True)
        self.bias = torch.autograd.Variable(torch.ones(2), requires_grad=True)

bert_ranking = BertForPassageRanking.from_pretrained("saved_models/duoBERT/",
                                                     from_tf=True)
type_embed_weight = bert_ranking.bert.embeddings.token_type_embeddings.weight.data
bert_ranking.bert.embeddings.token_type_embeddings.weight.data = torch.cat((type_embed_weight, torch.zeros(1,1024)))
bert_ranking.eval()
bert_ranking.cuda()
""

BertForPassageRanking(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, el

In [45]:
tokenizer = BertTokenizer("saved_models/monoBERT/vocab.txt")

In [61]:
tokenizer(text="a", text_pair='b', is_pretokenized=False,return_token_type_ids=True)

{'input_ids': [101, 1037, 102, 1038, 102], 'token_type_ids': [0, 0, 0, 1, 1], 'attention_mask': [1, 1, 1, 1, 1]}

In [91]:
query = 'how can I unfollow polaris 400 emails'
bad_passage = 'Best Answer: Plastics are used in wide range of things. So it is produced in a very huge amount and its convenience is undeniable. Recycling of plastic is very important because it is made from the oil which will cause the regular depletion of this limited resource.With the recycling of plastic we can save oil and can use it for longer time. Moreover recycling do not cause harm to the quality of plastics.est Answer: Plastics are used in wide range of things. So it is produced in a very huge amount and its convenience is undeniable. Recycling of plastic is very important because it is made from the oil which will cause the regular depletion of this limited resource.'
good_passage= "polaris 400 starter. Follow polaris 400 starter to get e-mail alerts and updates on your eBay Feed. Unfollow polaris 400 starter to stop getting updates on your eBay Feed.Yay! You're now following polaris 400 starter in your eBay Feed.ollow polaris 400 starter to get e-mail alerts and updates on your eBay Feed. Unfollow polaris 400 starter to stop getting updates on your eBay Feed. Yay! You're now following polaris 400 starter in your eBay Feed."

def custom_numericalize(query, docA, docB):
    query_ids = [tokenizer.cls_token_id] + tokenizer.encode(query, add_special_tokens=False) + [tokenizer.sep_token_id]
    query_token_type_ids = [0]*len(query_ids)

    docA_ids = tokenizer.encode(docA, add_special_tokens=False) + [tokenizer.sep_token_id]
    docA_token_type_ids = [1]*len(docA_ids)

    docB_ids = tokenizer.encode(docB, add_special_tokens=False) + [tokenizer.sep_token_id]
    docB_token_type_ids = [2]*len(docB_ids)

    input_ids = torch.tensor(query_ids+docA_ids+docB_ids).unsqueeze(0)
    input_type_ids = torch.tensor(query_token_type_ids+docA_token_type_ids+docB_token_type_ids).unsqueeze(0)
    return input_ids, input_type_ids

print(f"Input length={input_ids.shape}")

Input length=torch.Size([1, 272])


In [9]:
duoBERT_reranker_transform = DuoBERT_ReRanker_Transform("saved_models/duoBERT/", get_doc_fn)

Loading chekcpoint from saved_models/duoBERT/
DuoBERT ReRanker initialised on cuda. Batch size 32


In [11]:
test_samples = [{'query':"query text",'search_results':[("MARCO_32", 0.4), ("MARCO_55",0.3)]}]

test_samples = duoBERT_reranker_transform(test_samples)
test_samples

[{'query': 'query text',
  'search_results': [('MARCO_32', 0.4), ('MARCO_55', 0.3)],
  'reranked_results': [('MARCO_32', 4.047558307647705),
   ('MARCO_55', -4.047558307647705)]}]

In [71]:
outputs = bert_ranking(input_ids, token_type_ids=input_type_ids)
outputs

(tensor([[-0.0861,  0.6057]], grad_fn=<AddmmBackward>),)

In [75]:
outputs = bert_ranking(input_ids, token_type_ids=input_type_ids)
outputs

(tensor([[ 0.2233, -0.7396]], grad_fn=<AddmmBackward>),)

In [92]:
input_ids, input_type_ids = custom_numericalize(query, good_passage, bad_passage)
outputs = bert_ranking(input_ids.cuda(), token_type_ids=input_type_ids.cuda())
outputs

(tensor([[-0.2688,  0.3415]], device='cuda:0', grad_fn=<AddmmBackward>),)

In [93]:
input_ids, input_type_ids = custom_numericalize(query, bad_passage, good_passage)
outputs = bert_ranking(input_ids.cuda(), token_type_ids=input_type_ids.cuda())
outputs

(tensor([[ 0.5154, -0.5022]], device='cuda:0', grad_fn=<AddmmBackward>),)

In [47]:
input_ids = torch.tensor(tokenizer.encode("a [SEP] b")).unsqueeze(0)
input_ids

tensor([[  101,  1037,  1031, 19802,  1033,  1038,   102]])

In [65]:
tokenizer.decode(query_ids)

'why is it important to recycle plastic bags'

In [37]:
dummy_query = [
    'why is it important to recycle plastic bags',
    'why is it important to recycle plastic bags'
]

dummy_passage = [
    'Best Answer: Plastics are used in wide range of things. So it is produced in a very huge amount and its convenience is undeniable. Recycling of plastic is very important because it is made from the oil which will cause the regular depletion of this limited resource.With the recycling of plastic we can save oil and can use it for longer time. Moreover recycling do not cause harm to the quality of plastics.est Answer: Plastics are used in wide range of things. So it is produced in a very huge amount and its convenience is undeniable. Recycling of plastic is very important because it is made from the oil which will cause the regular depletion of this limited resource.',
    "polaris 400 starter. Follow polaris 400 starter to get e-mail alerts and updates on your eBay Feed. Unfollow polaris 400 starter to stop getting updates on your eBay Feed.Yay! You're now following polaris 400 starter in your eBay Feed.ollow polaris 400 starter to get e-mail alerts and updates on your eBay Feed. Unfollow polaris 400 starter to stop getting updates on your eBay Feed. Yay! You're now following polaris 400 starter in your eBay Feed."
]
bert_ranking.eval()
with torch.no_grad():
    for idx in range(len(dummy_query)):
        input_ids = torch.tensor(tokenizer.encode(text=dummy_query[idx], \
            text_pair=dummy_passage[idx], add_special_tokens=True)).unsqueeze(0)
        outputs = bert_ranking(input_ids)
        print(outputs)

(tensor([[ 0.0415, -0.0227]]),)
(tensor([[-0.9739,  0.0645]]),)


In [10]:
import tensorflow as tf

In [12]:
tf.compat.v1.one_hot([0,1,2,3], depth=2)

<tf.Tensor: id=10, shape=(4, 2), dtype=float32, numpy=
array([[1., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.]], dtype=float32)>

In [16]:
tf.matmul([[1,0]], [[2,3],
                    [4,5]])

<tf.Tensor: id=21, shape=(1, 2), dtype=int32, numpy=array([[2, 3]], dtype=int32)>

# Mono and Duo BERT tests

In [2]:
CAsT_raw_data_loader = CAsT_RawDataLoader()
get_query_fn = CAsT_raw_data_loader.get_query
get_doc_fn = CAsT_raw_data_loader.get_doc
eval_raw_samples = CAsT_raw_data_loader.get_topics("all")
CAsT_q_rels = CAsT_raw_data_loader.q_rels
len(eval_raw_samples)

NameError: name 'CAsT_RawDataLoader' is not defined

In [4]:
with jsonlines.open('saved_models/BART_Rewriter/BART_y2_self_rewrites.jsonl') as reader:
    rewritten_samples = []
    for sample_obj in reader:
        sample_obj["query"] = sample_obj["rewritten_query"]
        rewritten_samples.append(sample_obj)

In [50]:
rewritten_samples = [s for s in rewritten_samples if s["q_rel"]]
len(rewritten_samples)

163

In [70]:
for sample in rewritten_samples:
    if sample["q_rel"]:
        metric_sample = next(item for item in eval_BM25_results if item["q_id"] == sample["q_id"])
        print(f'MAP:{metric_sample["map"]:.3f}, MRR:{metric_sample["recip_rank"]:.3f}, NDCG:{metric_sample["ndcg_cut_3"]:.3f}, RECAL:{metric_sample["set_recall"]:.3f}')
    else:
        print("----")

MAP:0.458, MRR:1.000, NDCG:1.000, RECAL:0.952
MAP:0.290, MRR:1.000, NDCG:1.000, RECAL:0.515
MAP:0.101, MRR:0.167, NDCG:0.000, RECAL:0.575
MAP:0.379, MRR:1.000, NDCG:1.000, RECAL:0.758
MAP:0.007, MRR:0.007, NDCG:0.000, RECAL:1.000
MAP:0.069, MRR:0.500, NDCG:0.531, RECAL:0.417
MAP:0.001, MRR:0.032, NDCG:0.000, RECAL:0.023
MAP:0.054, MRR:0.500, NDCG:0.296, RECAL:0.519
MAP:0.046, MRR:0.038, NDCG:0.000, RECAL:1.000
MAP:0.012, MRR:0.045, NDCG:0.000, RECAL:0.286
MAP:0.312, MRR:0.500, NDCG:0.531, RECAL:1.000
MAP:0.478, MRR:1.000, NDCG:1.000, RECAL:0.981
MAP:0.574, MRR:1.000, NDCG:1.000, RECAL:0.929
MAP:0.166, MRR:0.333, NDCG:0.235, RECAL:1.000
MAP:0.009, MRR:0.045, NDCG:0.000, RECAL:0.235
MAP:0.374, MRR:1.000, NDCG:1.000, RECAL:1.000
MAP:1.000, MRR:1.000, NDCG:1.000, RECAL:1.000
MAP:0.515, MRR:1.000, NDCG:0.765, RECAL:1.000
MAP:0.421, MRR:0.500, NDCG:0.531, RECAL:1.000
MAP:0.024, MRR:0.020, NDCG:0.000, RECAL:1.000
MAP:0.756, MRR:1.000, NDCG:1.000, RECAL:1.000
MAP:0.000, MRR:0.000, NDCG:0.000, 

In [36]:
eval_samples = Query_Resolver_Transform(get_query_fn, utterance_type="manual_rewritten_utterance")(eval_raw_samples)
eval_BM25_results = BM25_Search_Transform(index_dir='datasets/TREC_CAsT/CAsT_collection_with_meta.index', 
                                          key_fields={'query_field':'query', 'target_field':'search_results'},
                                          hits=1000)(eval_samples)
expr = Ranking_Experiment(CAsT_q_rels)
print("BM25 results")
print(expr(eval_BM25_results))


BM25 results
{'map': 0.19751538242688302, 'recip_rank': 0.44835046104876053, 'ndcg_cut_3': 0.28929055308073365, 'set_recall': 0.8436929924957347}


In [37]:
run_file_exporter = RUN_File_Transform_Exporter('saved_models/BM25/y1_manual_BM25_1k.run', model_name='manual_BM25')
run_file_exporter(eval_rewritten_samples)

Successfully written 1000 samples from 1 queries run to: saved_models/BM25/y1_manual_BM25_1k.run


In [11]:
eval_BM25_results = Reranking_Flattener_Transform()(eval_BM25_results)

KeyboardInterrupt: 

In [161]:
eval_BM25_results = Document_Resolver_Transform(get_doc_fn)(eval_BM25_results)
eval_BM25_results = MonoBERT_Numericalise_Transform("saved_models/monoBERT/vocab.txt")(eval_BM25_results)

KeyboardInterrupt: 

In [12]:
monoBERT_transform = monoBERT_Scorer_Transform()

Loading chekcpoint from ./saved_models/monoBERT/
MonoBERT ReRanker initialised on device cpu. Batch size 32


In [164]:
small_samples = eval_BM25_results[:20]

In [145]:
test_samples = [
    {'q_id':'31_1', 'd_id':'CAR_2174ad0aa50712ff24035c23f59a3c2b43267650'}, # relevant
    {'q_id':'31_1', 'd_id':'MARCO_2182567'} # not relevant
]
test_samples = Query_Resolver_Transform(get_query_fn, utterance_type="manual_rewritten_utterance")(test_samples)
test_samples = Document_Resolver_Transform(get_doc_fn)(test_samples)
test_samples = MonoBERT_Numericalise_Transform("saved_models/monoBERT/vocab.txt")(test_samples)

In [149]:
monoBERT_results = monoBERT_transform(test_samples)

tensor([0.2858, 0.1724], device='cuda:0') torch.Size([2])



In [165]:
eval_duo_reranked_samples = RUN_File_Search_Transform('saved_models/duoBERT/BART_y1_BM25_mono+duoBERT@10_500.run', hits=500)(eval_BM25_results)

In [8]:
monoBERT_reranker_transform = MonoBERT_ReRanker_Transform('saved_models/monoBERT/', get_doc_fn, device="cuda:0", 
                                                          key_fields={'query_field':'query', 'target_field':'search_results'},
                                                          batch_size=100)

Loading chekcpoint from saved_models/monoBERT/
MonoBERT ReRanker initialised on device cuda:0. Batch size 100


In [21]:
eval_mono_reranked_samples = monoBERT_reranker_transform(eval_BM25_results)

In [169]:
# for sample in eval_mono_reranked_samples:
#     sample["search_results"] = [(d_id, score) for d_id, score in sample["reranked_results"]]
expr = Ranking_Experiment(CAsT_q_rels)
print("monoBERT results")
print(expr(eval_mono_reranked_samples))

monoBERT results
{'map': 0.24648819890078935, 'recip_rank': 0.5795464123485042, 'ndcg_cut_3': 0.44722112989468366, 'set_recall': 0.5273517857320486}


In [12]:
duoBERT_reranker_transform = DuoBERT_ReRanker_Transform("saved_models/duoBERT/", get_doc_fn, rerank_top=10)

Loading chekcpoint from saved_models/duoBERT/
DuoBERT ReRanker initialised on cuda. Batch size 32


In [23]:
eval_duo_reranked_samples = duoBERT_reranker_transform(eval_mono_reranked_samples)

KeyboardInterrupt: 

In [171]:
for sample in eval_mono_reranked_samples:
    sample["search_results"] = [(d_id, score) for d_id, score in sample["reranked_results"]]
expr = Ranking_Experiment(CAsT_q_rels)
print("DuoBERT results")
print(expr(eval_duo_reranked_samples))

DuoBERT results
{'map': 0.24652898926477615, 'recip_rank': 0.5882122619760941, 'ndcg_cut_3': 0.4675022330487291, 'set_recall': 0.5273517857320486}


In [22]:
run_file_exporter = RUN_File_Transform_Exporter('saved_models/monoBERT/BART_y2_BM25_monoBERT_1k_Relevance_Query_Expansion_x2.run', model_name='BART+monoBERT+RelQE_x2')
run_file_exporter(eval_mono_reranked_samples)

Successfully written 216000 samples from 216 queries run to: saved_models/monoBERT/BART_y2_BM25_monoBERT_1k_Relevance_Query_Expansion_x2.run


In [50]:
print(expr(eval_mono_reranked_samples))

{'map': 0.052117325670267264, 'recip_rank': 0.01098901098901099, 'ndcg_cut_3': 0.0, 'set_recall': 0.9523809523809523}
{'map': 0.010214954952767282, 'recip_rank': 0.0056179775280898875, 'ndcg_cut_3': 0.0, 'set_recall': 0.5151515151515151}
{'map': 0.06408624931028395, 'recip_rank': 0.01818181818181818, 'ndcg_cut_3': 0.0, 'set_recall': 0.675}
{'map': 0.04213950997777283, 'recip_rank': 0.011596268899639687, 'ndcg_cut_3': 0.0, 'set_recall': 0.7141774891774891}


In [34]:
for sample in eval_mono_reranked_samples:
    del sample['q_rel']
    del sample['prev_turns']
    del sample['q_id']
    del sample['reranked_results']

In [67]:
eval_mono_reranked_samples[3]

{'prev_turns': ['31_1', '31_2', '31_3'],
 'q_id': '31_4',
 'q_rel': ['CAR_10d3185b987e4a3247ddf302d1a77af0d8089057',
  'CAR_2345165208413ae4d82c1a5f4145564a1216b267',
  'CAR_2dfe7045bf79ea1c881c0fce663113cb941e1b8d',
  'CAR_6d9ed0c735dfc7b5e4c487b3582fa9e24adece7e',
  'CAR_6fd0c21b5771545db27842d1cbb35961bc78d548',
  'CAR_80b9307468496c2e2cda4d3f67c3424adf40a2d1',
  'CAR_abd3181f15bc33e8ef101761ef7f29d5d8943f87',
  'CAR_be26d2e44cbc807a3477aeba36d49931df11be1e',
  'CAR_dedad08c40164cee79464fdeb8da5e3e46b402ea',
  'CAR_fd096e610b37453cb360b8b45f90f876dac850d1',
  'CAR_ff289fa20c9d45ffdb79d31487034aaf513e027b',
  'MARCO_1127654',
  'MARCO_1198831',
  'MARCO_1734120',
  'MARCO_1736079',
  'MARCO_1801310',
  'MARCO_1801311',
  'MARCO_1826636',
  'MARCO_1855231',
  'MARCO_1916591',
  'MARCO_1916598',
  'MARCO_1976714',
  'MARCO_217417',
  'MARCO_217421',
  'MARCO_2539075',
  'MARCO_2700408',
  'MARCO_28054',
  'MARCO_28058',
  'MARCO_2916594',
  'MARCO_2916597',
  'MARCO_3368653',
  'MARCO_

In [69]:
get_doc_fn('MARCO_6575537')

'Iron poisoning causes abdominal pain, vomiting, diarrhea, lethargy, and dehydration. Non-small cell lung cancer is the most common type of lung cancer and can cause a cough, chest pain, and more. Carbon monoxide poisoning can be fatal, symptoms include headache, dizziness, nausea, and vomiting.'

In [32]:
import csv
keys = eval_raw_samples[0].keys()
with open('manual_y1_BM25_monoBERT_500.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(eval_raw_samples)

a
b
c


## Relevance Feedback query expansion

In [16]:
eval_duo_reranked_samples[4]

{'prev_turns': ['81_1', '81_2', '81_3', '81_4'],
 'q_id': '81_5',
 'q_rel': ['MARCO_7713538'],
 'unresolved_query': 'How do I choose a new one?',
 'previous_queries': ['How do you know when your garage door opener is going bad?',
  'Now it stopped working. Why?',
  'How much does it cost for someone to fix it?',
  'How about replacing it instead?'],
 'resolved_query': 'How do I choose a new garage door opener?',
 'full_rewritten_queries': [' How do you know when your garage door opener is going bad?',
  ' Now your garage Door opener stopped working. Why?',
  ' How much does it cost for someone to fix an opener that goes bad?',
  ' How about replacing an opener instead of replacing it instead of fixing it?',
  ' How do I choose a new opener when the old one stops working?'],
 'rewritten_query': ' How do I choose a new opener when the old one stops working?',
 'query': ' How do I choose a new opener when the old one stops working?',
 'search_results': [('MARCO_852601', -1.881676554679870

In [17]:
RM_samples = Relevance_Model_Transform(get_doc_fn, top_k=20)(eval_duo_reranked_samples)

In [18]:
expanded_samples = Simple_Query_Expansion_Transform(top_k=20)(RM_samples)

In [19]:
expanded_samples[4]

{'prev_turns': ['81_1', '81_2', '81_3', '81_4'],
 'q_id': '81_5',
 'q_rel': ['MARCO_7713538'],
 'unresolved_query': 'How do I choose a new one?',
 'previous_queries': ['How do you know when your garage door opener is going bad?',
  'Now it stopped working. Why?',
  'How much does it cost for someone to fix it?',
  'How about replacing it instead?'],
 'resolved_query': 'How do I choose a new garage door opener?',
 'full_rewritten_queries': [' How do you know when your garage door opener is going bad?',
  ' Now your garage Door opener stopped working. Why?',
  ' How much does it cost for someone to fix an opener that goes bad?',
  ' How about replacing an opener instead of replacing it instead of fixing it?',
  ' How do I choose a new opener when the old one stops working?'],
 'rewritten_query': ' How do I choose a new opener when the old one stops working?',
 'query': ' How do I choose a new opener when the old one stops working?',
 'search_results': [('MARCO_852601', -1.881676554679870

In [1]:
get_doc_fn('CAR_0aaa843df0bd467b96e5a496fc0b033d')

NameError: name 'get_doc_fn' is not defined

In [129]:
from sklearn.feature_extraction import text
text.ENGLISH_STOP_WORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

# Full Automatic pipe: mono, duo, relevanceQE, x2

In [30]:
with jsonlines.open('saved_models/BART_Rewriter/BART_y2_self_rewrites.jsonl') as reader:
    eval_rewritten_samples = []
    for sample_obj in reader:
        sample_obj["query"] = sample_obj["rewritten_query"]
        eval_rewritten_samples.append(sample_obj)
eval_rewritten_samples = eval_rewritten_samples[:1]

In [31]:
eval_rewritten_samples = BM25_Search_Transform(index_dir='datasets/TREC_CAsT/CAsT_collection_with_meta.index', 
                                          key_fields={'query_field':'query', 'target_field':'search_results'},
                                          hits=1000)(eval_rewritten_samples)


monoBERT_reranker_transform = MonoBERT_ReRanker_Transform('saved_models/monoBERT/', get_doc_fn, device="cuda:1", 
                                                          key_fields={'query_field':'query', 'target_field':'search_results'},
                                                          batch_size=100)
eval_rewritten_samples = monoBERT_reranker_transform(eval_rewritten_samples)
run_file_exporter = RUN_File_Transform_Exporter('saved_models/monoBERT/y2_automatic_BART_BM25_monoBERT_1k.run', model_name='BART+monoBERT')
run_file_exporter(eval_rewritten_samples)


duoBERT_reranker_transform = DuoBERT_ReRanker_Transform("saved_models/duoBERT/", get_doc_fn, device="cuda:1", rerank_top=10)
eval_rewritten_samples = duoBERT_reranker_transform(eval_rewritten_samples)
for sample in eval_rewritten_samples:
    sample["search_results"] = [(d_id, score) for d_id, score in sample["reranked_results"]]
run_file_exporter = RUN_File_Transform_Exporter('saved_models/duoBERT/y2_automatic_BART_BM25_mono_duoBERT_1k.run', model_name='BART+mono+duoBERT')
run_file_exporter(eval_rewritten_samples)


eval_rewritten_samples = Relevance_Model_Transform(get_doc_fn, top_k=20)(eval_rewritten_samples)
eval_rewritten_samples = Simple_Query_Expansion_Transform(top_k=20)(eval_rewritten_samples)


eval_rewritten_samples = BM25_Search_Transform(index_dir='datasets/TREC_CAsT/CAsT_collection_with_meta.index', 
                                          key_fields={'query_field':'expanded_query', 'target_field':'search_results'},
                                          hits=1000)(eval_rewritten_samples)


monoBERT_reranker_transform = MonoBERT_ReRanker_Transform('saved_models/monoBERT/', get_doc_fn, device="cuda:1", 
                                                          key_fields={'query_field':'query', 'target_field':'search_results'},
                                                          batch_size=100)
eval_rewritten_samples = monoBERT_reranker_transform(eval_rewritten_samples)
run_file_exporter = RUN_File_Transform_Exporter('saved_models/monoBERT/y2_automatic_BART_BM25_monoBERT_Rel_Query_Exp_x2_1k.run', model_name='BART+monoBERT_Rel_Query_Exp_x2')
run_file_exporter(eval_rewritten_samples)


duoBERT_reranker_transform = DuoBERT_ReRanker_Transform("saved_models/duoBERT/", get_doc_fn, device="cuda:1", rerank_top=10)
eval_rewritten_samples = duoBERT_reranker_transform(eval_rewritten_samples)
for sample in eval_rewritten_samples:
    sample["search_results"] = [(d_id, score) for d_id, score in sample["reranked_results"]]
run_file_exporter = RUN_File_Transform_Exporter('saved_models/duoBERT/y2_automatic_BART_BM25_mono_duoBERT_Rel_Query_Exp_x2_1k.run', model_name='BART+mono+duoBERT_Rel_Query_Exp_x2')
run_file_exporter(eval_rewritten_samples)


Loading chekcpoint from saved_models/monoBERT/
MonoBERT ReRanker initialised on device cuda:1. Batch size 100


Successfully written 1000 samples from 1 queries run to: saved_models/monoBERT/y2_automatic_BART_BM25_monoBERT_1k.run
Loading chekcpoint from saved_models/duoBERT/
DuoBERT ReRanker initialised on cuda:1. Batch size 32


Successfully written 1000 samples from 1 queries run to: saved_models/duoBERT/y2_automatic_BART_BM25_mono_duoBERT_1k.run



Loading chekcpoint from saved_models/monoBERT/
MonoBERT ReRanker initialised on device cuda:1. Batch size 100


Successfully written 1000 samples from 1 queries run to: saved_models/monoBERT/y2_automatic_BART_BM25_monoBERT_Rel_Query_Exp_x2_1k.run
Loading chekcpoint from saved_models/duoBERT/
DuoBERT ReRanker initialised on cuda:1. Batch size 32


Successfully written 1000 samples from 1 queries run to: saved_models/duoBERT/y2_automatic_BART_BM25_mono_duoBERT_Rel_Query_Exp_x2_1k.run


In [29]:
eval_rewritten_samples[1]

{'prev_turns': ['81_1'],
 'q_id': '81_2',
 'q_rel': ['MARCO_3942603'],
 'unresolved_query': 'Now it stopped working. Why?',
 'previous_queries': ['How do you know when your garage door opener is going bad?'],
 'resolved_query': 'Now my garage door opener stopped working. Why?',
 'full_rewritten_queries': [' How do you know when your garage door opener is going bad?',
  ' Now your garage Door opener stopped working. Why?'],
 'rewritten_query': ' Now your garage Door opener stopped working. Why?',
 'query': ' Now your garage Door opener stopped working. Why?',
 'search_results': [('CAR_e24797700a9c01bf0aa53a913ae8727ae275d9e6',
   39.54975289106369),
  ('MARCO_5822849', 24.210433304309845),
  ('MARCO_4516825', 23.484754860401154),
  ('MARCO_5498474', 14.138743788003922),
  ('MARCO_4688655', 3.464914470911026),
  ('MARCO_1104225', 1.9280383586883545),
  ('MARCO_4516819', -9.984885036945343),
  ('MARCO_8752378', -11.942408055067062),
  ('MARCO_3942603', -15.249652236700058),
  ('MARCO_6115